## Deep Learning Keras Model

In [1]:
import time
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('instacart_pca.csv')
df.head()

,order_id,product_id,aisle_id,orders,order_number,days_reorder_ratio,days_since_prior_order,n_orders,order_hour_of_day,reorder_interval,total_items,reordered
0,1,0.0,0.0,0.0,4,9.000000,0.0,0.0,0.0,1.000000,0.0,1.0
1,1,1.0,1.0,76.0,4,0.473684,0.0,10.0,0.0,19.000000,11.0,1.0
2,1,2.0,2.0,4.0,4,0.552632,0.0,14.0,0.0,16.285714,31.0,0.0
3,1,3.0,2.0,6.0,4,0.750000,0.0,12.0,0.0,12.000000,0.0,0.0
4,1,4.0,3.0,22.0,4,0.529412,0.0,5.0,0.0,17.000000,0.0,1.0


In [3]:
df.shape

(1384617, 12)

In [4]:
X = df.drop(['reordered'], axis=1)
y = df['reordered'].values.reshape(-1,1)
(X.shape, y.shape)

((1384617, 11), (1384617, 1))

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
(X_train.shape, y_train.shape)

((969231, 11), (969231, 1))

In [7]:
from kerastuner import HyperModel, RandomSearch
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import log_loss, accuracy_score, classification_report, confusion_matrix

## Keras Tuner Hypermodel

In [8]:
class DLModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        model.add(
            Dense(
                units=hp.Int(
                    'dense_units_1',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128),
                activation="relu",
                input_shape=self.input_shape,
                  )
            )
        model.add(
            Dropout(
                rate=hp.Float(
                    "dropout_1", min_value=0.0, max_value=0.5, default=0.25, step=0.05,
                )
            )
        )
        model.add(BatchNormalization())
        model.add(
            Dense(
                units=hp.Int(
                    "dense_units_2", 
                    min_value=32, 
                    max_value=512, 
                    step=32, 
                    default=64
                ),
                activation="relu",
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    "dropout_2", min_value=0.0, max_value=0.5, default=0.25, step=0.05
                )
            )
        )
        model.add(BatchNormalization())
        model.add(Dense(self.num_classes, activation="sigmoid"))

        model.compile(
            optimizer=Adam(
                hp.Float(
                    "learning_rate",
                    min_value=1e-4,
                    max_value=1e-2,
                    default=1e-3,
                )
            ),
            loss="binary_crossentropy",
            metrics=["accuracy"],
        )

        return model

In [9]:
SEED = 101

NUM_CLASSES = y_train.shape[1]
INPUT_SHAPE = (X_train.shape[1],)

N_EPOCH_SEARCH = 10
MAX_TRIALS = 5
EXECUTION_PER_TRIAL = 2

hypermodel = DLModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(
        hypermodel,
        objective='val_accuracy',
        seed=SEED,
        max_trials=MAX_TRIALS, 
        executions_per_trial=EXECUTION_PER_TRIAL, 
        directory='/content/',
        project_name='dl_model',
    )

search_start = time.time()
tuner.search(X_train, y_train, epochs=N_EPOCH_SEARCH, validation_split=0.2)
search_end = time.time()
elapsed_time = search_end - search_start

tuner.results_summary()

dlm = tuner.get_best_models(num_models=1)[0]
dlm.summary()

loss, accuracy = dlm.evaluate(X_test, y_test)

print(f"Elapsed time = {elapsed_time:10.4f} s, accuracy = {accuracy}, loss = {loss}")

Epoch 1/10
24231/24231 [==============================] - 153s 6ms/step - loss: 0.6761 - accuracy: 0.5967 - val_loss: 0.7639 - val_accuracy: 0.6040
Epoch 2/10
24231/24231 [==============================] - 156s 6ms/step - loss: 0.6706 - accuracy: 0.6022 - val_loss: 0.9535 - val_accuracy: 0.6076
Epoch 3/10
24231/24231 [==============================] - 151s 6ms/step - loss: 0.6699 - accuracy: 0.6022 - val_loss: 2.2486 - val_accuracy: 0.6020
Epoch 4/10
24231/24231 [==============================] - 158s 7ms/step - loss: 0.6703 - accuracy: 0.6014 - val_loss: 1.0167 - val_accuracy: 0.6043
Epoch 5/10
24231/24231 [==============================] - 156s 6ms/step - loss: 0.6692 - accuracy: 0.6024 - val_loss: 1.8053 - val_accuracy: 0.6055
Epoch 6/10
24231/24231 [==============================] - 155s 6ms/step - loss: 0.6686 - accuracy: 0.6034 - val_loss: 5.9112 - val_accuracy: 0.6056
Epoch 7/10
24231/24231 [==============================] - 147s 6ms/step - loss: 0.6693 - accuracy: 0.6027 - val_

Epoch 1/10
24231/24231 [==============================] - 81s 3ms/step - loss: 0.6704 - accuracy: 0.6018 - val_loss: 0.7055 - val_accuracy: 0.6047
Epoch 2/10
24231/24231 [==============================] - 85s 3ms/step - loss: 0.6676 - accuracy: 0.6037 - val_loss: 0.7085 - val_accuracy: 0.6073
Epoch 3/10
24231/24231 [==============================] - 82s 3ms/step - loss: 0.6673 - accuracy: 0.6036 - val_loss: 0.9539 - val_accuracy: 0.6043
Epoch 4/10
24231/24231 [==============================] - 79s 3ms/step - loss: 0.6672 - accuracy: 0.6050 - val_loss: 0.6778 - val_accuracy: 0.6046
Epoch 5/10
24231/24231 [==============================] - 84s 3ms/step - loss: 0.6670 - accuracy: 0.6050 - val_loss: 0.7347 - val_accuracy: 0.6014
Epoch 6/10
24231/24231 [==============================] - 85s 4ms/step - loss: 0.6688 - accuracy: 0.6036 - val_loss: 0.8558 - val_accuracy: 0.6079
Epoch 7/10
24231/24231 [==============================] - 86s 4ms/step - loss: 0.6673 - accuracy: 0.6049 - val_loss: 1

Epoch 1/10
24231/24231 [==============================] - 77s 3ms/step - loss: 0.6706 - accuracy: 0.6016 - val_loss: 0.6844 - val_accuracy: 0.6069
Epoch 2/10
24231/24231 [==============================] - 78s 3ms/step - loss: 0.6676 - accuracy: 0.6033 - val_loss: 0.7392 - val_accuracy: 0.6041
Epoch 3/10
24231/24231 [==============================] - 76s 3ms/step - loss: 0.6669 - accuracy: 0.6038 - val_loss: 0.6944 - val_accuracy: 0.6026
Epoch 4/10
24231/24231 [==============================] - 81s 3ms/step - loss: 0.6666 - accuracy: 0.6042 - val_loss: 0.7355 - val_accuracy: 0.6050
Epoch 5/10
24231/24231 [==============================] - 79s 3ms/step - loss: 0.6662 - accuracy: 0.6036 - val_loss: 0.7176 - val_accuracy: 0.6083
Epoch 6/10
24231/24231 [==============================] - 84s 3ms/step - loss: 0.6661 - accuracy: 0.6042 - val_loss: 0.8753 - val_accuracy: 0.6017
Epoch 7/10
24231/24231 [==============================] - 75s 3ms/step - loss: 0.6654 - accuracy: 0.6034 - val_loss: 1

Epoch 1/10
24231/24231 [==============================] - 142s 6ms/step - loss: 0.6721 - accuracy: 0.6002 - val_loss: 1.0395 - val_accuracy: 0.6049
Epoch 2/10
24231/24231 [==============================] - 139s 6ms/step - loss: 0.6682 - accuracy: 0.6034 - val_loss: 0.6904 - val_accuracy: 0.6042
Epoch 3/10
24231/24231 [==============================] - 137s 6ms/step - loss: 0.6680 - accuracy: 0.6033 - val_loss: 1.6588 - val_accuracy: 0.6060
Epoch 4/10
24231/24231 [==============================] - 134s 6ms/step - loss: 0.6677 - accuracy: 0.6033 - val_loss: 0.7560 - val_accuracy: 0.6056
Epoch 5/10
24231/24231 [==============================] - 139s 6ms/step - loss: 0.6672 - accuracy: 0.6035 - val_loss: 0.8150 - val_accuracy: 0.6083
Epoch 6/10
24231/24231 [==============================] - 145s 6ms/step - loss: 0.6664 - accuracy: 0.6035 - val_loss: 0.8364 - val_accuracy: 0.6077
Epoch 7/10
24231/24231 [==============================] - 145s 6ms/step - loss: 0.6664 - accuracy: 0.6038 - val_

Epoch 1/10
24231/24231 [==============================] - 59s 2ms/step - loss: 0.6748 - accuracy: 0.5981 - val_loss: 0.6762 - val_accuracy: 0.5981
Epoch 2/10
24231/24231 [==============================] - 60s 2ms/step - loss: 0.6742 - accuracy: 0.5988 - val_loss: 0.6854 - val_accuracy: 0.5981
Epoch 3/10
24231/24231 [==============================] - 58s 2ms/step - loss: 0.6741 - accuracy: 0.5988 - val_loss: 0.6749 - val_accuracy: 0.5981
Epoch 4/10
24231/24231 [==============================] - 56s 2ms/step - loss: 0.6741 - accuracy: 0.5988 - val_loss: 0.9342 - val_accuracy: 0.5980
Epoch 5/10
24231/24231 [==============================] - 58s 2ms/step - loss: 0.6740 - accuracy: 0.5988 - val_loss: 0.6817 - val_accuracy: 0.6022
Epoch 6/10
24231/24231 [==============================] - 62s 3ms/step - loss: 0.6735 - accuracy: 0.5992 - val_loss: 0.6941 - val_accuracy: 0.5981
Epoch 7/10
24231/24231 [==============================] - 62s 3ms/step - loss: 0.6718 - accuracy: 0.6005 - val_loss: 0

INFO:tensorflow:Oracle triggered exit


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 416)               4992      
_________________________________________________________________
dropout (Dropout)            (None, 416)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 416)               1664      
_________________________________________________________________
dense_1 (Dense)              (None, 384)               160128    
_________________________________________________________________
dropout_1 (Dropout)          (None, 384)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 384)               1536      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3

In [10]:
prediction_dlm = (dlm.predict(X_test) > 0.5).astype("int32")
print("Confusion Matrix:\n", confusion_matrix(y_test,prediction_dlm))
print("\nClassification Report:\n", classification_report(y_test,prediction_dlm))
print("Accuracy Score: ", accuracy_score(y_test,prediction_dlm))

Confusion Matrix:
 [[ 27704 139121]
 [ 23315 225246]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.54      0.17      0.25    166825
         1.0       0.62      0.91      0.73    248561

    accuracy                           0.61    415386
   macro avg       0.58      0.54      0.49    415386
weighted avg       0.59      0.61      0.54    415386

Accuracy Score:  0.6089516738647908


In [12]:
dlm.save('dlm_pca.h5')